# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
# Your code here
import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [3]:
# Your code here
cur.execute("""
               SELECT firstName, officeCode, lastName, city, employeeNumber
               FROM employees e
               JOIN offices o
               USING(officeCode)  
               WHERE o.city == "Boston"
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,firstName,officeCode,lastName,city,employeeNumber
0,Julie,2,Firrelli,Boston,1188
1,Steve,2,Patterson,Boston,1216


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [4]:
# Your code here
cur.execute("""SELECT o.city, COUNT(e.employeeNumber) AS num_employees 
               FROM offices AS o 
               LEFT JOIN employees AS e
               USING(officeCode)
               GROUP BY officeCode
               HAVING num_employees == 0;
               """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df



# Didn't seem to work
# ACTUALLY IT DID.  The officeCode is different, but still BOSTON

,city,num_employees
0,Boston,0


## Write 3 Questions of your own and answer them

In [5]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold

In [6]:
# Your code here
# Display the last 10 orders for a salesrep 1188  (From previous problem)
cur.execute("""SELECT *
               FROM orders AS o
               LEFT JOIN customers as c
               USING(customerNumber)
               WHERE SalesRepEmployeeNumber == 1188
               ORDER BY OrderDate
               DESC
               LIMIT 10
               """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,10369,2005-01-20,2005-01-28,2005-01-24,Shipped,,379,Collectables For Less Inc.,Nelson,Allen,6175558555,7825 Douglas Av.,,Brickhaven,MA,58339,USA,1188,70700
1,10365,2005-01-07,2005-01-18,2005-01-11,Shipped,,320,Mini Creations Ltd.,Huang,Wing,5085559555,4575 Hillside Dr.,,New Bedford,MA,50553,USA,1188,94500
2,10307,2004-10-14,2004-10-23,2004-10-20,Shipped,,339,"Classic Gift Ideas, Inc",Cervantes,Francisca,2155554695,782 First Street,,Philadelphia,PA,71270,USA,1188,81100
3,10294,2004-09-10,2004-09-17,2004-09-14,Shipped,,204,Online Mini Collectables,Barajas,Miguel,6175557555,7635 Spinnaker Dr.,,Brickhaven,MA,58339,USA,1188,68700
4,10276,2004-08-02,2004-08-11,2004-08-08,Shipped,,204,Online Mini Collectables,Barajas,Miguel,6175557555,7635 Spinnaker Dr.,,Brickhaven,MA,58339,USA,1188,68700
5,10274,2004-07-21,2004-07-29,2004-07-22,Shipped,,379,Collectables For Less Inc.,Nelson,Allen,6175558555,7825 Douglas Av.,,Brickhaven,MA,58339,USA,1188,70700
6,10249,2004-05-08,2004-05-17,2004-05-11,Shipped,Can we deliver the new Ford Mustang models by ...,173,Cambridge Collectables Co.,Tseng,Jerry,6175555555,4658 Baden Av.,,Cambridge,MA,51247,USA,1188,43400
7,10243,2004-04-26,2004-05-03,2004-04-28,Shipped,,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,,Boston,MA,51003,USA,1188,85100
8,10228,2004-03-10,2004-03-18,2004-03-13,Shipped,,173,Cambridge Collectables Co.,Tseng,Jerry,6175555555,4658 Baden Av.,,Cambridge,MA,51247,USA,1188,43400
9,10207,2003-12-09,2003-12-17,2003-12-11,Shipped,Check on availability.,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,,Boston,MA,51003,USA,1188,85100


In [7]:
# Your code here
# Who is the leading salesperson alltime (not sure how to do last year??)
cur.execute("""SELECT e.firstName, e.lastName, e.officeCode, SUM(p.amount) AS sales_total
               FROM payments AS p
               LEFT JOIN customers AS c
               USING(customerNumber)
               LEFT JOIN employees AS e
               ON e.employeeNumber == c.salesRepEmployeeNumber
               GROUP BY e.employeeNumber
               ORDER BY sales_total
               DESC;
                """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,firstName,lastName,officeCode,sales_total
0,Gerard,Hernandez,4,1112003.81
1,Leslie,Jennings,1,989906.55
2,Pamela,Castillo,4,750201.87
3,Larry,Bott,7,686653.25
4,Barry,Jones,7,637672.65
5,George,Vanauf,3,584406.80
6,Loui,Bondur,4,569485.75
7,Andy,Fixter,6,509385.82
8,Peter,Marsh,6,497907.16
9,Foon Yue,Tseng,3,488212.67


In [8]:
# Your code here
# Who has the most customers?

cur.execute("""SELECT COUNT(e.employeeNumber) AS number_customers, e.firstName, e.lastname
               FROM employees as e
               JOIN customers as c
               ON e.employeeNumber == c.salesRepEmployeeNumber
               GROUP BY e.employeeNumber
               ORDER BY number_customers
               DESC
               """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,number_customers,firstName,lastName
0,10,Pamela,Castillo
1,9,Barry,Jones
2,8,George,Vanauf
3,8,Larry,Bott
4,7,Foon Yue,Tseng
5,7,Gerard,Hernandez
6,6,Leslie,Jennings
7,6,Leslie,Thompson
8,6,Julie,Firrelli
9,6,Steve,Patterson


## Level Up: Display the names of every individual product that each employee has sold

In [12]:
# Your code here
cur.execute("""
SELECT * 
FROM employees AS e
JOIN customers AS c
ON e.employeeNumber == c.salesRepEmployeeNumber
JOIN orders AS o
USING(customerNumber)
JOIN orderdetails AS od
USING(orderNumber)
JOIN products AS p
USING(productCode)
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,customerNumber,customerName,...,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep,124,Mini Gifts Distributors Ltd.,...,121.64,2,1958 Setra Bus,Trucks and Buses,1:12,Welly Diecast Productions,"Model features 30 windows, skylights & glare r...",1579,77.90,136.67
1,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep,124,Mini Gifts Distributors Ltd.,...,101.50,4,1940 Ford Pickup Truck,Trucks and Buses,1:18,Studio M Art Models,"This model features soft rubber tires, working...",2613,58.33,116.67
2,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep,124,Mini Gifts Distributors Ltd.,...,43.27,3,1939 Cadillac Limousine,Vintage Cars,1:18,Studio M Art Models,Features completely detailed interior includin...,6645,23.14,50.31
3,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep,124,Mini Gifts Distributors Ltd.,...,58.82,1,1996 Peterbilt 379 Stake Bed with Outrigger,Trucks and Buses,1:32,Red Start Diecast,"This model features, opening doors, detailed e...",814,33.61,64.64
4,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep,124,Mini Gifts Distributors Ltd.,...,173.17,7,1968 Ford Mustang,Classic Cars,1:12,Autoart Studio Design,"Hood, doors and trunk all open to reveal highl...",68,95.34,194.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991,1702,Gerard,Martin,x2312,mgerard@classicmodelcars.com,4,1102,Sales Rep,484,"Iberia Gift Imports, Corp.",...,52.49,8,1954 Greyhound Scenicruiser,Trucks and Buses,1:32,Classic Metal Creations,"Model features bi-level seating, 50 windows, s...",2874,25.98,54.11
2992,1702,Gerard,Martin,x2312,mgerard@classicmodelcars.com,4,1102,Sales Rep,484,"Iberia Gift Imports, Corp.",...,59.03,1,1950's Chicago Surface Lines Streetcar,Trains,1:32,Gearbox Collectibles,This streetcar is a joy to see. It has 80 sepa...,8601,26.72,62.14
2993,1702,Gerard,Martin,x2312,mgerard@classicmodelcars.com,4,1102,Sales Rep,484,"Iberia Gift Imports, Corp.",...,92.60,4,Diamond T620 Semi-Skirted Tanker,Trucks and Buses,1:50,Highway 66 Mini Classics,This limited edition model is licensed and per...,1016,68.29,115.75
2994,1702,Gerard,Martin,x2312,mgerard@classicmodelcars.com,4,1102,Sales Rep,484,"Iberia Gift Imports, Corp.",...,56.91,2,1911 Ford Town Car,Vintage Cars,1:18,Motor City Art Classics,"Features opening hood, opening doors, opening ...",540,33.30,60.54


## Level Up: Display the Number of Products each employee has sold

In [14]:
# Your code here
cur.execute("""
SELECT e.employeeNumber, e.lastName, e.firstName, COUNT(productCode) AS numProductsSold
FROM employees AS e
JOIN customers AS c
ON e.employeeNumber == c.salesRepEmployeeNumber
JOIN orders AS o
USING(customerNumber)
JOIN orderdetails AS od
USING(orderNumber)
JOIN products AS p
USING(productCode)
GROUP BY e.employeeNumber
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,lastName,firstName,numProductsSold
0,1165,Jennings,Leslie,331
1,1166,Thompson,Leslie,114
2,1188,Firrelli,Julie,124
3,1216,Patterson,Steve,152
4,1286,Tseng,Foon Yue,142
5,1323,Vanauf,George,211
6,1337,Bondur,Loui,177
7,1370,Hernandez,Gerard,396
8,1401,Castillo,Pamela,272
9,1501,Bott,Larry,236


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!